# T5-DMLM(train-sciq-passage-level) Text2Text Generation on Sciq
使用 Sciq dataset訓練 T5 Distractor Generation<br>
直接使用 trainer 訓練 <br>

### GPU

In [1]:
!nvidia-smi

Fri Aug 25 16:11:47 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|


|   0  NVIDIA TITAN RTX               On  | 00000000:09:00.0 Off |                  N/A |
| 41%   39C    P8              35W / 280W |      1MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX               On  | 00000000:0A:00.0 Off |                  N/A |
| 41%   36C    P8              32W / 280W |      1MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
                                                                                         
+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   G

In [2]:
project_name = "test on T5 with T5"
import os

os.environ["WANDB_PROJECT"] = project_name
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### import

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

### Loading the dataset

In [4]:
from datasets import load_dataset

dataset = load_dataset("sciq")

Using custom data configuration default
Reusing dataset sciq (/user_data/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 11679
    })
    validation: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
})

In [6]:
dataset['train'][0]

{'question': 'What type of organism is commonly used in preparation of foods such as cheese and yogurt?',
 'distractor3': 'viruses',
 'distractor1': 'protozoa',
 'distractor2': 'gymnosperms',
 'correct_answer': 'mesophilic organisms',
 'support': 'Mesophiles grow best in moderate temperature, typically between 25°C and 40°C (77°F and 104°F). Mesophiles are often found living in or on the bodies of humans or other animals. The optimal growth temperature of many pathogenic mesophiles is 37°C (98°F), the normal human body temperature. Mesophilic organisms have important uses in food preparation, including cheese, yogurt, beer and wine.'}

In [7]:
train = dataset['train']
valid = dataset['validation']
test = dataset['test']

In [8]:
train = list(train)
test = list(test)
valid = list(valid)

In [9]:
from sklearn.utils import shuffle

train = shuffle(train, random_state=777)
len(train), len(valid)

(11679, 1000)

減少training data 對結果的影響

In [10]:
train = train[:5000]

In [11]:
len(train), len(valid), len(test)

(8000, 1000, 1000)

In [12]:
train[0]

{'question': 'In the presence of oxygen, hydrogen can interact to make what?',
 'distractor3': 'acid',
 'distractor1': 'carbon',
 'distractor2': 'helium',
 'correct_answer': 'water',
 'support': 'A pile of leaves slowly rots in the backyard. In the presence of oxygen, hydrogen can interact to make water. Gold can be stretched into very thin wires.'}

### Prepare data

In [13]:
def processData(data):
    
    sentences = []
    labels = []
    answers = []
    for d in data:
        sentence = d['question']
        distractors = [d['distractor1'], d['distractor2'], d['distractor3']]
        answer = d['correct_answer']
        
        # 避免dataset的label有空白
        distractors = [dis.strip() for dis in distractors]
        
        sentences.append(sentence)
        labels.append('_ of distractors are ' + ', '.join(distractors))
        answers.append(answer)
        
    return sentences, answers, labels

In [14]:
train_sent, train_answer, train_label = processData(train)
valid_sent, valid_answer, valid_label = processData(valid)
test_sent, test_answer, test_label = processData(test)

In [15]:
print(test_label[0])

_ of distractors are antioxidants, Oxygen, residues


In [16]:
for l in test_label:
    if 'ultraviolet light' in l:
        print(l)

_ of distractors are invisible light, sunlight, ultraviolet light


In [17]:
len(train_sent), len(train_answer), len(train_label)

(8000, 8000, 8000)

In [18]:
for idx in range(2):
    print(train_sent[idx])
    print(train_answer[idx])
    print(train_label[idx])
    print()

In the presence of oxygen, hydrogen can interact to make what?
water
_ of distractors are carbon, helium, acid

What type of diagnosis happens before a baby is born?
prenatal
_ of distractors are maternal, fetal, postnatal



In [19]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

In [20]:
train_encodings = tokenizer(train_sent, train_answer,truncation=True, padding=True)
valid_encodings = tokenizer(valid_sent, valid_answer,truncation=True, padding=True)
test_encodings = tokenizer(test_sent, test_answer,truncation=True, padding=True)

In [21]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [22]:
print(train_encodings.input_ids[0])

[86, 8, 3053, 13, 11035, 6, 20913, 54, 6815, 12, 143, 125, 58, 1, 387, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [23]:
tokenizer.decode(train_encodings.input_ids[0])

'In the presence of oxygen, hydrogen can interact to make what?</s> water</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

In [24]:
def add_labels(encodings, distractors):
    
    distractors_encodings = tokenizer(distractors, padding=True)
    labels = []
    for i in range(len(distractors_encodings.input_ids)):
        labels.append(distractors_encodings.input_ids[i])
    
    encodings["labels"] = labels
    return encodings

In [25]:
train_encodings = add_labels(train_encodings, train_label)
valid_encodings = add_labels(valid_encodings, valid_label)
test_encodings = add_labels(test_encodings, test_label)

In [26]:
len(train_encodings.input_ids)

8000

In [27]:
class SciqDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SciqDataset(train_encodings)
valid_dataset = SciqDataset(valid_encodings)
test_dataset = SciqDataset(test_encodings)

In [28]:
len(train_dataset), len(valid_dataset), len(test_dataset)

(8000, 1000, 1000)

### Fine-tuning

In [29]:
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

model = T5ForConditionalGeneration.from_pretrained("t5-base")
model.resize_token_embeddings(len(tokenizer))

Embedding(32100, 768)

In [30]:
model_dict = torch.nn.ModuleDict({
    'model': model,
})
checkpoint = torch.load('/user_data/Cloze/dtt_mask_lm_model/mcq_all_passage_level/epoch=03-dev_loss=0.12.ckpt')
model_dict.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [31]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    output_dir = "./results-0",
    save_strategy = "epoch",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=50,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="P@1",
    weight_decay=0.01,
    predict_with_generate=True,
    eval_accumulation_steps = 1,
    report_to="wandb" if os.getenv("WANDB_PROJECT") else "none"
)

In [32]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [33]:
import numpy as np
def compute_metrics(p):
    predictions, labels = p
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # store all article
    predicted = []
    true_label = []
    
    for k in range(len(decoded_labels)):
        pred = decoded_preds[k]
        label = decoded_labels[k]

        pred_list = pred.split(', ')
        label_list = label.split(', ')
        
        pred_list[0] = pred_list[0].split('are ')[-1]
        label_list[0] = label_list[0].split('are ')[-1]

        predicted.append(pred_list)
        true_label.append(label_list)

    # evaluation metrics
    p1 = 0
    p3 = 0
    r3 = 0
    f3 = 0
    for idx in range(len(true_label)):
        distractors = predicted[idx]
        labels = true_label[idx]

        act_set = set(labels)
        pred1_set = set(distractors[:1])
        pred3_set = set(distractors[:3])

        p_1 = len(act_set & pred1_set) / float(1)
        p_3 = len(act_set & pred3_set) / float(3)
        r_3 = len(act_set & pred3_set) / float(len(act_set))

        if p_3 == 0 and r_3 == 0:
            f1_3 = 0
        else:
            f1_3 = 2 * (p_3 * r_3 / (p_3 + r_3))

        p1+=p_1
        p3+=p_3
        r3+=r_3
        f3+=f1_3

    avg_p1 = p1 / len(true_label)
    avg_p3 = p3 / len(true_label)
    avg_r3 = r3 / len(true_label)
    avg_f3 = f3 / len(true_label)

    result = {'P@1': avg_p1,
              'P@3': avg_p3,
              'R@3': avg_r3,
              'F1@3': avg_f3}
    
    return result

In [34]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [35]:
trainer.train()

/user_data/.local/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8000
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 12500


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ms0004284. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,P@1,P@3,R@3,F1@3
1,No log,0.525311,0.102000,0.044333,0.044333,0.044289
2,0.450900,0.514264,0.116000,0.052333,0.052310,0.052267
3,0.450900,0.507920,0.133000,0.060000,0.060000,0.059956
4,0.377000,0.505112,0.150000,0.076000,0.076000,0.075956
5,0.377000,0.504551,0.155000,0.082333,0.082667,0.082422
6,0.346100,0.506584,0.181000,0.087667,0.087643,0.087511
7,0.346100,0.509594,0.188000,0.095667,0.095810,0.095578
8,0.322200,0.510521,0.185000,0.095333,0.095476,0.095244
9,0.322200,0.515437,0.187000,0.097000,0.096976,0.096844
10,0.301100,0.519799,0.194000,0.107667,0.107976,0.107689


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
Saving model checkpoint to ./results-0/checkpoint-250
Configuration saved in ./results-0/checkpoint-250/config.json
Model weights saved in ./results-0/checkpoint-250/pytorch_model.bin
tokenizer config file saved in ./results-0/checkpoint-250/tokenizer_config.json
Special tokens file saved in ./results-0/checkpoint-250/special_tokens_map.json
/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
Saving model checkpoint to ./results-0/checkpoint-500
Configuration saved in ./results-0/checkpoint-500/config.json
Model weights saved in ./results-0/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results-0/checkpoint-500

TrainOutput(global_step=12500, training_loss=0.2191597833251953, metrics={'train_runtime': 8626.3091, 'train_samples_per_second': 46.37, 'train_steps_per_second': 1.449, 'total_flos': 5.2808067072e+16, 'train_loss': 0.2191597833251953, 'epoch': 50.0})

In [36]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.6122552156448364,
 'eval_P@1': 0.215,
 'eval_P@3': 0.13633333333333328,
 'eval_R@3': 0.13666666666666663,
 'eval_F1@3': 0.1364222222222222,
 'eval_runtime': 31.1138,
 'eval_samples_per_second': 32.14,
 'eval_steps_per_second': 1.028,
 'epoch': 50.0}

In [37]:
trainer.save_model('/user_data/CTG/train/DG/parameter_analysis/mcq_all_finetuning_on_sciq/sciq_5000/model/t5-base-text2text-sciq-pretrain-on-mcq-all-passage-level-5000')

Saving model checkpoint to /user_data/CTG/train/DG/parameter_analysis/mcq_all_finetuning_on_sciq/sciq_8000/model/t5-base-text2text-sciq-pretrain-on-mcq-all-passage-level-8000
Configuration saved in /user_data/CTG/train/DG/parameter_analysis/mcq_all_finetuning_on_sciq/sciq_8000/model/t5-base-text2text-sciq-pretrain-on-mcq-all-passage-level-8000/config.json
Model weights saved in /user_data/CTG/train/DG/parameter_analysis/mcq_all_finetuning_on_sciq/sciq_8000/model/t5-base-text2text-sciq-pretrain-on-mcq-all-passage-level-8000/pytorch_model.bin
tokenizer config file saved in /user_data/CTG/train/DG/parameter_analysis/mcq_all_finetuning_on_sciq/sciq_8000/model/t5-base-text2text-sciq-pretrain-on-mcq-all-passage-level-8000/tokenizer_config.json
Special tokens file saved in /user_data/CTG/train/DG/parameter_analysis/mcq_all_finetuning_on_sciq/sciq_8000/model/t5-base-text2text-sciq-pretrain-on-mcq-all-passage-level-8000/special_tokens_map.json


In [38]:
predictions, labels, metrics = trainer.predict(valid_dataset)
print('valid: ')
metrics

***** Running Prediction *****
  Num examples = 1000
  Batch size = 16


valid: 


{'test_loss': 0.6122552156448364,
 'test_P@1': 0.215,
 'test_P@3': 0.13633333333333328,
 'test_R@3': 0.13666666666666663,
 'test_F1@3': 0.1364222222222222,
 'test_runtime': 30.4716,
 'test_samples_per_second': 32.817,
 'test_steps_per_second': 1.05}

In [39]:
predictions, labels, metrics = trainer.predict(test_dataset)
print('test: ')
metrics

***** Running Prediction *****
  Num examples = 1000
  Batch size = 16


test: 


{'test_loss': 0.7293282747268677,
 'test_P@1': 0.231,
 'test_P@3': 0.15533333333333327,
 'test_R@3': 0.15484126984126975,
 'test_F1@3': 0.15486666666666662,
 'test_runtime': 32.1349,
 'test_samples_per_second': 31.119,
 'test_steps_per_second': 0.996}

In [40]:
import json
def write_json(data, path):
    
    jsonString = json.dumps(data)
    jsonFile = open(path, "w")
    jsonFile.write(jsonString)
    jsonFile.close()

In [41]:
def save_data(data, predictions, labels, file_name):
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # store all article
    predicted = []
    true_label = []
    
    for k in range(len(decoded_labels)):
        pred = decoded_preds[k]
        label = decoded_labels[k]

        pred_list = pred.split(', ')
        label_list = label.split(', ')
        
        pred_list[0] = pred_list[0].split('are ')[-1]
        label_list[0] = label_list[0].split('are ')[-1]

        predicted.append(pred_list)
        true_label.append(label_list)
    
    
    # evaluation metrics
    for idx in range(len(true_label)):
        distractors = predicted[idx]
        labels = true_label[idx]
        
        data[idx]['pred_distractors'] = distractors

        act_set = set(labels)
        pred1_set = set(distractors[:1])
        pred3_set = set(distractors[:3])

        p_1 = len(act_set & pred1_set) / float(1)
        p_3 = len(act_set & pred3_set) / float(3)
        r_3 = len(act_set & pred3_set) / float(len(act_set))

        if p_3 == 0 and r_3 == 0:
            f1_3 = 0
        else:
            f1_3 = 2 * (p_3 * r_3 / (p_3 + r_3))
            
        data[idx]['metric'] = {'P@1': p_1, 'P@3': p_3, 'R@3': r_3, 'F1@3': f1_3}
        
    write_json(data, file_name)
    print(file_name + ' is saved :)')

In [42]:
save_data(test, predictions, labels, '/user_data/CTG/test_result/sciq_test_t5_text2text_pretrain_on_mcq_all_passage_level_5000.json')

/user_data/CTG/test_result/sciq_test_t5_text2text_pretrain_on_mcq_all_passage_level_8000.json is saved :)


In [43]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

tokenizer = T5Tokenizer.from_pretrained("/user_data/CTG/model/t5-base-text2text-sciq-pretrain-on-sciq-train-passage-level-dtt-retrieve-8000")
model = T5ForConditionalGeneration.from_pretrained("/user_data/CTG/model/t5-base-text2text-sciq-pretrain-on-sciq-train-passage-level-dtt-retrieve-8000")

Didn't find file /user_data/CTG/model/t5-base-text2text-sciq-pretrain-on-sciq-train-passage-level-dtt-retrieve-8000/added_tokens.json. We won't load it.
loading file /user_data/CTG/model/t5-base-text2text-sciq-pretrain-on-sciq-train-passage-level-dtt-retrieve-8000/spiece.model
loading file None
loading file /user_data/CTG/model/t5-base-text2text-sciq-pretrain-on-sciq-train-passage-level-dtt-retrieve-8000/special_tokens_map.json
loading file /user_data/CTG/model/t5-base-text2text-sciq-pretrain-on-sciq-train-passage-level-dtt-retrieve-8000/tokenizer_config.json
loading configuration file /user_data/CTG/model/t5-base-text2text-sciq-pretrain-on-sciq-train-passage-level-dtt-retrieve-8000/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,


In [44]:
batch_size = 64
args = Seq2SeqTrainingArguments(
    output_dir = "./results",
    save_strategy = "epoch",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=50,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="P@1",
    weight_decay=0.01,
    predict_with_generate=True,
    eval_accumulation_steps = 1,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [45]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [46]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [47]:
predictions, labels, metrics = trainer.predict(valid_dataset)
print('valid: ')
metrics

***** Running Prediction *****
  Num examples = 1000
  Batch size = 64
/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


valid: 


{'test_loss': 0.8240611553192139,
 'test_P@1': 0.185,
 'test_P@3': 0.11733333333333297,
 'test_R@3': 0.11783333333333297,
 'test_F1@3': 0.11753333333333298,
 'test_runtime': 25.0027,
 'test_samples_per_second': 39.996,
 'test_steps_per_second': 0.32}

In [48]:
test_predictions, test_labels, test_metrics = trainer.predict(test_dataset)
test_metrics

***** Running Prediction *****
  Num examples = 1000
  Batch size = 64


{'test_loss': 1.0030803680419922,
 'test_P@1': 0.198,
 'test_P@3': 0.1173333333333331,
 'test_R@3': 0.11745238095238072,
 'test_F1@3': 0.11726666666666644,
 'test_runtime': 27.1839,
 'test_samples_per_second': 36.786,
 'test_steps_per_second': 0.294}

Result

In [49]:
import json
def read_data(path):
    with open(path) as f:
        data = json.load(f)
    return data

In [50]:
test = read_data('/user_data/CTG/test_result/sciq_test_t5_text2text_pretrain_on_sciq_training_set_passage_level_dtt_retrieve_8000.json')

In [51]:
for i in range(0, 100, 7):
    example = test[i]
    sentence = example['question']
    answer = example['correct_answer']
    distractors = [example['distractor1'], example['distractor2'], example['distractor3']]
    pred_distractors = example['pred_distractors']
    metric = example['metric']
    
    print('question:', sentence.replace('**blank**', '_'))
    print('answer:', answer)
    print('distractors:', distractors)
    print('predict:', pred_distractors)
    print('metric:', metric)
    print()

question: Compounds that are capable of accepting electrons, such as o 2 or f2, are called what?
answer: oxidants
distractors: ['antioxidants', 'Oxygen', 'residues']
predict: ['oxides', 'particles', 'anions']
metric: {'P@1': 0.0, 'P@3': 0.0, 'R@3': 0.0, 'F1@3': 0}

question: Which type of tree is dominant in temperate forests?
answer: deciduous
distractors: ['vines', 'fungus', 'shrubs']
predict: ['perennial', 'fibrous', 'coniferous']
metric: {'P@1': 0.0, 'P@3': 0.0, 'R@3': 0.0, 'F1@3': 0}

question: Only about one percent of plants have lost what ability, turning them into consumers and even predators, instead of producers?
answer: photosynthesis
distractors: ['flowering', 'rooting', 'growth']
predict: ['glycolysis', 'atherosclerosis', 'absorption']
metric: {'P@1': 0.0, 'P@3': 0.0, 'R@3': 0.0, 'F1@3': 0}

question: Presence of a cell wall, large central vacuole, and organelles called plastids distinguish what type of cell?
answer: plant
distractors: ['animal', 'reproductive', 'heterotr